In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_1samp

# 1 Initial Data
### 1.1 Import all datasets

#### Original data

In [2]:
sales = pd.read_csv("data/umsatzdaten_gekuerzt.csv")
weather = pd.read_csv("data/wetter.csv")
kiwo = pd.read_csv("data/kiwo.csv")
test_ids = pd.read_csv("data/test.csv")

# Convert all date columns to datetime format
sales["Datum"] = pd.to_datetime(sales["Datum"])
weather["Datum"] = pd.to_datetime(weather["Datum"])
kiwo["Datum"] = pd.to_datetime(kiwo["Datum"])
test_ids["Datum"] = pd.to_datetime(test_ids["Datum"])

#### Economic indicators

In [3]:
# Load the dataset 
cpi_url = 'https://www.destatis.de/static/de_/opendata/data/verbraucherpreisindex_gesamtindex_bv41.csv'
gdp_url = 'https://www.destatis.de/static/de_/opendata/data/bruttoinlandsprodukt_originalwert.csv'
unemp_url = 'https://www.destatis.de/static/de_/opendata/data/arbeitslosenquote_deutschland_originalwert.csv'

# Read the economic data from the URLs
cpi = pd.read_csv(cpi_url, sep=';', skiprows=1)
gdp = pd.read_csv(gdp_url, sep=';', skiprows=1)
unemp = pd.read_csv(unemp_url, sep=';', encoding='latin-1', skiprows=1, on_bad_lines='skip')

# Process the data
cpi['Month'] = pd.to_datetime(cpi['Datum'], format='%d/%m/%Y').dt.to_period('M').dt.start_time
unemp['Month'] = pd.to_datetime(unemp['Datum'], format='%d/%m/%Y').dt.to_period('M').dt.start_time
gdp['Quarter'] = pd.to_datetime(gdp['Datum'], format='%d/%m/%Y').dt.to_period('Q').dt.start_time

# Keep the first 3 columns and rename them
cpi = cpi[['Month', 'Originalwert, 2020=100']]
gdp = gdp[['Quarter', 'in jeweiligen Preisen, Mrd. EUR, Originalwert']]
unemp = unemp[['Month', 'Arbeitslosenquote aller zivilen Erwerbspersonen, insgesamt in %']]

# Rename the columns
cpi.columns = ['Month', 'CPI']
gdp.columns = ['Quarter', 'GDP']
unemp.columns = ['Month', 'Unemployment']

In [4]:
# Merge dataframes
econ = pd.merge(cpi, unemp, on='Month', how='outer')
econ['Quarter'] = econ['Month'].dt.to_period('Q').dt.start_time
econ = pd.merge(econ, gdp, on='Quarter', how='outer')
# Keep the rows where date is between 01-01-2012 and 01-08-2019
econ = econ[(econ['Month'] >= '2012-01-01') & (econ['Month'] <= '2019-08-01')]

# Convert object to float64 
econ['GDP'] = econ['GDP'].str.replace(',', '.').astype(float)
econ['CPI'] = econ['CPI'].str.replace(',', '.').astype(float)
econ['Unemployment'] = econ['Unemployment'].str.replace(',', '.').astype(float)

### 1.2 Merge all the dataframes step by step
One dataframe merges sales with weather and kiwo.

A second dataframe merges the test ids with weather and kiwo.

Both get concatenated to `feature_collection`. Here we can collect all additional features we want in our model.

We will split the feature df later into training, validation and test data.

In [5]:
weather

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
0,2012-01-01,8.0,9.8250,14,58.0
1,2012-01-02,7.0,7.4375,12,NaN
2,2012-01-03,8.0,5.5375,18,63.0
3,2012-01-04,4.0,5.6875,19,80.0
4,2012-01-05,6.0,5.3000,23,80.0
...,...,...,...,...,...
2596,2019-07-28,3.0,23.3500,14,5.0
2597,2019-07-29,6.0,25.2500,7,61.0
2598,2019-07-30,7.0,20.7375,8,61.0
2599,2019-07-31,6.0,20.4500,7,61.0


## Fill the nan in the weather data 
For this we use the ERA5 reanalysis data

total_wind: Wind speed at 2 m height, calculated from u10 and v10, in km/h

t2m: Air temperature at 2 m height, in degrees Celsius (°C)

d2m: Dew point temperature at 2 m height, in degrees Celsius (°C)

msl: Mean sea level pressure, in hectopascals (hPa)

tp: Total precipitation since the previous time step, in millimeters (mm)

In [6]:
era5 = pd.read_csv("data/ERA5-Hourly.csv")
era5['Datum'] = pd.to_datetime(era5['time'])
era5 = era5.set_index('Datum')

era5['t2m'] = era5['t2m'] - 273.15                          # K → °C
era5['d2m'] = era5['d2m'] - 273.15                          # K → °C
era5['msl'] = era5['msl'] / 100                            # Pa → hPa
era5['tp'] = era5['tp'] * 1000                             # m → mm

era5['tp'] = era5['tp'].where(era5['tp'] >= 0.1, 0)

era5_daily_mean = era5[['t2m', 'd2m', 'msl', 'total_wind']].resample('D').mean()

era5_daily_tp = era5['tp'].resample('D').sum()

era5_daily = pd.concat([era5_daily_mean, era5_daily_tp], axis=1).reset_index()

print(era5_daily)


          Datum        t2m        d2m          msl  total_wind         tp
0    2012-01-01   8.162089   7.685473  1003.999354    5.439706   8.486000
1    2012-01-02   6.914612   5.679782  1002.981331    5.436736   5.152987
2    2012-01-03   5.202969   3.135344   999.926733    9.409089  12.619905
3    2012-01-04   5.139351   2.798372   998.188107    9.481877   5.161377
4    2012-01-05   4.986291   2.718407   977.934627    9.414261  12.603815
...         ...        ...        ...          ...         ...        ...
2765 2019-07-28  21.204862  17.259328  1005.000895    5.178551   0.581403
2766 2019-07-29  21.498980  18.630335  1009.221642    2.333303   3.538525
2767 2019-07-30  19.256249  15.935728  1011.628986    2.616957   5.484295
2768 2019-07-31  18.223670  16.445438  1014.256088    2.091335  19.800271
2769 2019-08-01  18.066264  14.719160  1016.149031    1.792455   0.129602

[2770 rows x 6 columns]


In [7]:
merged = pd.merge(era5_daily,weather, on='Datum', how='left')

# Fehlende Werte ersetzen
merged['Bewoelkung'] = merged['Bewoelkung'].fillna(6)
merged['Temperatur'] = merged['Temperatur'].fillna(merged['t2m'])
merged['Windgeschwindigkeit'] = merged['Windgeschwindigkeit'].fillna(merged['total_wind'])
merged['Wettercode'] = merged['Wettercode'].fillna(0)
merged['Datum'] = merged['Datum'].fillna(merged['Datum'])

# Optional: nicht mehr benötigte Spalten entfernen
weather = merged.drop(columns=[ 't2m', 'total_wind','d2m','msl','tp'])
weather['Rain'] = merged['tp']
weather 

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain
0,2012-01-01,8.0,9.8250,14.0,58.0,8.486000
1,2012-01-02,7.0,7.4375,12.0,0.0,5.152987
2,2012-01-03,8.0,5.5375,18.0,63.0,12.619905
3,2012-01-04,4.0,5.6875,19.0,80.0,5.161377
4,2012-01-05,6.0,5.3000,23.0,80.0,12.603815
...,...,...,...,...,...,...
2765,2019-07-28,3.0,23.3500,14.0,5.0,0.581403
2766,2019-07-29,6.0,25.2500,7.0,61.0,3.538525
2767,2019-07-30,7.0,20.7375,8.0,61.0,5.484295
2768,2019-07-31,6.0,20.4500,7.0,61.0,19.800271


In [8]:
# 1. Merge sales data with weather data
merged = pd.merge(sales, weather, on="Datum", how="left")

# 2. Merge with Kieler Woche data
merged = pd.merge(merged, kiwo, on="Datum", how="left")
merged["KielerWoche"] = merged["KielerWoche"].fillna(0).astype(int)

# 3. Merge test data with weather and Kieler Woche data
test_merged = pd.merge(test_ids, weather, on="Datum", how="left")
test_merged = pd.merge(test_merged, kiwo, on="Datum", how="left")
test_merged["KielerWoche"] = test_merged["KielerWoche"].fillna(0).astype(int)

# 4. Add empty sales column to test dataset
test_merged["Umsatz"] = pd.NA

# 5. Combine training and test data into one DataFrame
final_merged = pd.concat([merged, test_merged], ignore_index=True)

# 6. Merge with economic data
final_merged['Month'] = final_merged['Datum'].dt.to_period('M').dt.start_time
feature_collection = pd.merge(final_merged, econ, on='Month', how='left') 

# 6. Make id the index of the dataframe
#feature_collection = feature_collection.set_index("id")
feature_collection

/tmp/ipykernel_5112/1840747790.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_merged = pd.concat([merged, test_merged], ignore_index=True)


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain,KielerWoche,Month,CPI,Unemployment,Quarter,GDP
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,1.567431,0,2013-07-01,93.5,6.8,2013-07-01,728.23
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,0.0,0.140967,0,2013-07-01,93.5,6.8,2013-07-01,728.23
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,6.276774,0,2013-07-01,93.5,6.8,2013-07-01,728.23
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0.126180,0,2013-07-01,93.5,6.8,2013-07-01,728.23
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,0.0,1.255988,0,2013-07-01,93.5,6.8,2013-07-01,728.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,6,NaN,8.0,4.3000,4.0,0.0,0.392234,0,2018-12-01,98.5,4.9,2018-10-01,881.52
11160,1812236,2018-12-23,6,NaN,7.0,6.4500,9.0,61.0,4.244212,0,2018-12-01,98.5,4.9,2018-10-01,881.52
11161,1812246,2018-12-24,6,NaN,7.0,2.5000,10.0,22.0,0.000000,0,2018-12-01,98.5,4.9,2018-10-01,881.52
11162,1812276,2018-12-27,6,NaN,7.0,7.1250,12.0,20.0,0.000000,0,2018-12-01,98.5,4.9,2018-10-01,881.52


In [9]:
feature_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11164 entries, 0 to 11163
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   11164 non-null  int64         
 1   Datum                11164 non-null  datetime64[ns]
 2   Warengruppe          11164 non-null  int64         
 3   Umsatz               9334 non-null   float64       
 4   Bewoelkung           11164 non-null  float64       
 5   Temperatur           11164 non-null  float64       
 6   Windgeschwindigkeit  11164 non-null  float64       
 7   Wettercode           11164 non-null  float64       
 8   Rain                 11164 non-null  float64       
 9   KielerWoche          11164 non-null  int64         
 10  Month                11164 non-null  datetime64[ns]
 11  CPI                  11164 non-null  float64       
 12  Unemployment         11164 non-null  float64       
 13  Quarter              11164 non-

# 2 Feature Engineering
In this section we add aditional features to our initial dataset. 
### 2.1 Weather Categorisation
Meteorological data can be highly variable and noisy. Therefore, it can be useful to categorize weather variables into broader classes (e.g., "Rain", "Sunny") to reduce noise and make patterns more interpretable for the model. This approach helps to generalize the effect of weather on sales and can improve the robustness of the predictions.

In [10]:
def group_weather(code):
    if pd.isna(code):
        return "Other"
    try:
        code = int(code)
    except:
        return "Other"

    if code in range(50, 69):  # Drizzle, rain, freezing rain, sleet
        return "Rain"
    if code in range(20, 29):  # After rain
        return "After_Rain"
    elif code in range(80, 85):  # Showers, sleet showers
        return "Showers_and_Thunderstorms"
    elif code in range(85, 91):  # Snow showers, hail showers
        return "Snow_and_Ice"
    elif code in range(91, 100):  # Thunderstorms
        return "Showers_and_Thunderstorms"
    elif code in list(range(10, 13)) + list(range(40, 50)):  # Haze, fog
        return "Fog"
    elif code in range(70, 80):  # Continuous snowfall, ice needles, snow grains, etc.
        return "Snow_and_Ice"
    elif code in range(66, 69):  # Sleet
        return "Snow_and_Ice"
    elif code in range(76, 80):  # Ice needles, snow grains, ice pellets
        return "Snow_and_Ice"
    else:
        return "Other"
    
def temperature_class(temp):
    if pd.isna(temp):
        return "Unknown_temp"
    elif temp < 5:
        return "cold"
    elif temp < 15:
        return "cool"
    elif temp < 25:
        return "mild"
    else:
        return "warm"

def bewoelkung_klasse(value):
    if pd.isna(value):
        return "Unknown_cloud"
    elif value <= 6:
        return "sunny"
    else:
        return "cloudy"

def windklasse(wind):
    if pd.isna(wind):
        return "Unknown_wind"
    elif wind < 10:
        return "breeze"
    elif wind < 20:
        return "wind"
    else:
        return "storm"

In [11]:
feature_collection["Weathercategory"] = feature_collection["Wettercode"].apply(group_weather)
feature_collection["Temperatureclass"] = feature_collection["Temperatur"].apply(temperature_class)
feature_collection["Cloudclass"] = feature_collection["Bewoelkung"].apply(bewoelkung_klasse)
feature_collection["Windclass"] = feature_collection["Windgeschwindigkeit"].apply(windklasse)
print(feature_collection)

            id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0      1307011 2013-07-01            1  148.828353         6.0     17.8375   
1      1307021 2013-07-02            1  159.793757         3.0     17.3125   
2      1307031 2013-07-03            1  111.885594         7.0     21.0750   
3      1307041 2013-07-04            1  168.864941         7.0     18.8500   
4      1307051 2013-07-05            1  171.280754         5.0     19.9750   
...        ...        ...          ...         ...         ...         ...   
11159  1812226 2018-12-22            6         NaN         8.0      4.3000   
11160  1812236 2018-12-23            6         NaN         7.0      6.4500   
11161  1812246 2018-12-24            6         NaN         7.0      2.5000   
11162  1812276 2018-12-27            6         NaN         7.0      7.1250   
11163  1812286 2018-12-28            6         NaN         7.0      7.3125   

       Windgeschwindigkeit  Wettercode      Rain  KielerWoche  

### Rain during business hours

In [12]:
# Regenmenge zwischen 08:00 und 18:00 Uhr je Tag summieren
rain_8_to_18 = (
    era5.between_time('08:00', '18:00')['tp']
    .resample('D')
    .sum()
    .rename('Rain_Sum_8to18')
)
rain_8_to_18


feature_collection = feature_collection.merge(rain_8_to_18, on='Datum', how='left')
feature_collection['Rain_8to18'] = (feature_collection['Rain_Sum_8to18'] > 1).astype(int)
feature_collection

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain,KielerWoche,...,CPI,Unemployment,Quarter,GDP,Weathercategory,Temperatureclass,Cloudclass,Windclass,Rain_Sum_8to18,Rain_8to18
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,1.567431,0,...,93.5,6.8,2013-07-01,728.23,After_Rain,mild,sunny,wind,1.442231,1
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,0.0,0.140967,0,...,93.5,6.8,2013-07-01,728.23,Other,mild,sunny,wind,0.140967,0
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,6.276774,0,...,93.5,6.8,2013-07-01,728.23,Rain,mild,cloudy,breeze,5.542491,1
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0.126180,0,...,93.5,6.8,2013-07-01,728.23,After_Rain,mild,cloudy,breeze,0.126180,0
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,0.0,1.255988,0,...,93.5,6.8,2013-07-01,728.23,Other,mild,sunny,wind,1.255988,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,6,NaN,8.0,4.3000,4.0,0.0,0.392234,0,...,98.5,4.9,2018-10-01,881.52,Other,cold,cloudy,breeze,0.243416,0
11160,1812236,2018-12-23,6,NaN,7.0,6.4500,9.0,61.0,4.244212,0,...,98.5,4.9,2018-10-01,881.52,Rain,cool,cloudy,breeze,1.981295,1
11161,1812246,2018-12-24,6,NaN,7.0,2.5000,10.0,22.0,0.000000,0,...,98.5,4.9,2018-10-01,881.52,After_Rain,cold,cloudy,wind,0.000000,0
11162,1812276,2018-12-27,6,NaN,7.0,7.1250,12.0,20.0,0.000000,0,...,98.5,4.9,2018-10-01,881.52,After_Rain,cool,cloudy,wind,0.000000,0


In [13]:
# Rolling Mittelwert
temp_rolling = weather['Temperatur'].rolling(window=5, min_periods=1).mean()

# Platzhalter für Ergebnis
significantly_above = []
significantly_below = []

# Signifikanztest über Fenster
for i in range(len(weather)):
    if i < 5:
        significantly_above.append(0)
        significantly_below.append(0)
        continue

    window = weather['Temperatur'].iloc[i-5:i]
    current = weather['Temperatur'].iloc[i]

    t_stat, p_val = ttest_1samp(window, current)

    if p_val < 0.01 and current > window.mean():
        significantly_above.append(1)
        significantly_below.append(0)
    elif p_val < 0.01 and current < window.mean():
        significantly_above.append(0)
        significantly_below.append(1)
    else:
        significantly_above.append(0)
        significantly_below.append(0)

# Neuen Feature-DataFrame erstellen
weather_temp_features = pd.DataFrame({
    'Datum': weather['Datum'],
    'Temp_5d_avg': temp_rolling,
    'Temp_above_avg': significantly_above,
    'Temp_below_avg': significantly_below
})

weather_temp_features

feature_collection = feature_collection.merge(weather_temp_features, on='Datum', how='left')

# Tageshöchsttemperatur aus stündlichen Daten berechnen
tmax = (
    era5['t2m']
    .resample('D')
    .max()
    .rename('Tmax')
)

# Zur feature_collection hinzufügen
feature_collection = feature_collection.merge(tmax, on='Datum', how='left')

# Binäres Feature: Ist die Tageshöchsttemperatur über 25 °C?
feature_collection['Tmax_gt25'] = (feature_collection['Tmax'] > 25).astype(int)


# Neues Kombi-Feature: warm (≥25 °C) und stark nass (>5 mm)
feature_collection['Hot_HeavyRain'] = (
    (feature_collection['Temperatur'] >= 25) & 
    (feature_collection['Rain'] > 5)
).astype(int)

feature_collection

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain,KielerWoche,...,Cloudclass,Windclass,Rain_Sum_8to18,Rain_8to18,Temp_5d_avg,Temp_above_avg,Temp_below_avg,Tmax,Tmax_gt25,Hot_HeavyRain
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,1.567431,0,...,sunny,wind,1.442231,1,14.4575,1,0,18.199883,0,0
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,0.0,0.140967,0,...,sunny,wind,0.140967,0,15.4100,0,0,18.319065,0,0
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,6.276774,0,...,cloudy,breeze,5.542491,1,17.0150,1,0,22.473013,0,0
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0.126180,0,...,cloudy,breeze,0.126180,0,17.8175,0,0,19.594244,0,0
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,0.0,1.255988,0,...,sunny,wind,1.255988,1,19.0100,0,0,20.406262,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,6,NaN,8.0,4.3000,4.0,0.0,0.392234,0,...,cloudy,breeze,0.243416,0,4.7575,0,0,5.505522,0,0
11160,1812236,2018-12-23,6,NaN,7.0,6.4500,9.0,61.0,4.244212,0,...,cloudy,breeze,1.981295,1,4.9850,0,0,6.818994,0,0
11161,1812246,2018-12-24,6,NaN,7.0,2.5000,10.0,22.0,0.000000,0,...,cloudy,wind,0.000000,0,4.8500,0,0,4.503302,0,0
11162,1812276,2018-12-27,6,NaN,7.0,7.1250,12.0,20.0,0.000000,0,...,cloudy,wind,0.000000,0,6.0700,0,0,7.492173,0,0


### 2.2 Day of the Week and Weekend
Information about the day of the week and whether it is a weekend is relevant for the sales prediction model, because purchasing behaviour and sales differ (see Dataset Characteristics). So we add new features for the day of the week and a weekend flag.

In [14]:
feature_collection["Wochentag"] = feature_collection["Datum"].dt.weekday
feature_collection["Wochenende"] = feature_collection["Wochentag"].isin([5, 6]).astype(int)
feature_collection["Monat"] = feature_collection["Datum"].dt.month
feature_collection

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain,KielerWoche,...,Rain_8to18,Temp_5d_avg,Temp_above_avg,Temp_below_avg,Tmax,Tmax_gt25,Hot_HeavyRain,Wochentag,Wochenende,Monat
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,1.567431,0,...,1,14.4575,1,0,18.199883,0,0,0,0,7
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,0.0,0.140967,0,...,0,15.4100,0,0,18.319065,0,0,1,0,7
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,6.276774,0,...,1,17.0150,1,0,22.473013,0,0,2,0,7
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0.126180,0,...,0,17.8175,0,0,19.594244,0,0,3,0,7
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,0.0,1.255988,0,...,1,19.0100,0,0,20.406262,0,0,4,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,6,NaN,8.0,4.3000,4.0,0.0,0.392234,0,...,0,4.7575,0,0,5.505522,0,0,5,1,12
11160,1812236,2018-12-23,6,NaN,7.0,6.4500,9.0,61.0,4.244212,0,...,1,4.9850,0,0,6.818994,0,0,6,1,12
11161,1812246,2018-12-24,6,NaN,7.0,2.5000,10.0,22.0,0.000000,0,...,0,4.8500,0,0,4.503302,0,0,0,0,12
11162,1812276,2018-12-27,6,NaN,7.0,7.1250,12.0,20.0,0.000000,0,...,0,6.0700,0,0,7.492173,0,0,3,0,12


### 2.3 Public and School Holidays

In [15]:
public = pd.read_csv("data/public_holidays.csv")
public["Datum"] = pd.to_datetime(public["Datum"])
school = pd.read_csv("data/school_holidays.csv")
school["Datum"] = pd.to_datetime(school["Datum"])

# if statement for protection from multiple merges in the notebook
if "Feiertag" not in feature_collection.columns:
    feature_collection = pd.merge(feature_collection, public.drop(columns=['Celebration']), on="Datum", how="left")
    feature_collection["Feiertag"] = feature_collection["Feiertag"].fillna(0).astype(int)

if "Schulferien" not in feature_collection.columns:
    feature_collection = pd.merge(feature_collection, school.drop(columns=['Wochentag']), on="Datum", how="left")

feature_collection.info()
feature_collection

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11164 entries, 0 to 11163
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   11164 non-null  int64         
 1   Datum                11164 non-null  datetime64[ns]
 2   Warengruppe          11164 non-null  int64         
 3   Umsatz               9334 non-null   float64       
 4   Bewoelkung           11164 non-null  float64       
 5   Temperatur           11164 non-null  float64       
 6   Windgeschwindigkeit  11164 non-null  float64       
 7   Wettercode           11164 non-null  float64       
 8   Rain                 11164 non-null  float64       
 9   KielerWoche          11164 non-null  int64         
 10  Month                11164 non-null  datetime64[ns]
 11  CPI                  11164 non-null  float64       
 12  Unemployment         11164 non-null  float64       
 13  Quarter              11164 non-

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Rain,KielerWoche,...,Temp_above_avg,Temp_below_avg,Tmax,Tmax_gt25,Hot_HeavyRain,Wochentag,Wochenende,Monat,Feiertag,Schulferien
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,1.567431,0,...,1,0,18.199883,0,0,0,0,7,0,1
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,0.0,0.140967,0,...,0,0,18.319065,0,0,1,0,7,0,1
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,6.276774,0,...,1,0,22.473013,0,0,2,0,7,0,1
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0.126180,0,...,0,0,19.594244,0,0,3,0,7,0,1
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,0.0,1.255988,0,...,0,0,20.406262,0,0,4,0,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,6,NaN,8.0,4.3000,4.0,0.0,0.392234,0,...,0,0,5.505522,0,0,5,1,12,0,1
11160,1812236,2018-12-23,6,NaN,7.0,6.4500,9.0,61.0,4.244212,0,...,0,0,6.818994,0,0,6,1,12,0,1
11161,1812246,2018-12-24,6,NaN,7.0,2.5000,10.0,22.0,0.000000,0,...,0,0,4.503302,0,0,0,0,12,0,1
11162,1812276,2018-12-27,6,NaN,7.0,7.1250,12.0,20.0,0.000000,0,...,0,0,7.492173,0,0,3,0,12,0,1


### 2.99 "Residual" Category
This category marks oddities in the data.

All New Year's Eves have exceptionally high sales in product group 5 (Cake), which can be explained by the custom to eat Berliner (Pfannkuchen). But this occurs only 5 times, too few to form a group of their own.

The only other exceptionally high sales value appears on Monday 2014-05-05 in product group 2 (Roll). We suspect an error in the data.

In [16]:
feature_collection["Residual"] = 0
feature_collection.loc[feature_collection["id"] == 1312315, "Residual"] = 1
feature_collection.loc[feature_collection["id"] == 1412315, "Residual"] = 1
feature_collection.loc[feature_collection["id"] == 1512315, "Residual"] = 1
feature_collection.loc[feature_collection["id"] == 1612315, "Residual"] = 1
feature_collection.loc[feature_collection["id"] == 1712315, "Residual"] = 1
feature_collection.loc[feature_collection["id"] == 1405052, "Residual"] = 1
print(feature_collection[feature_collection["Residual"] == 1]["id"])

2119    1405052
7403    1312315
7760    1412315
8120    1512315
8476    1612315
8833    1712315
Name: id, dtype: int64


# 3 Feature Preparation
For the final training dataframe we need to replace all strings with numerical data.

### 3.1 Encoding
#### Label Encoding
Label encoding means that each category gets a label, for example: cold -> 0, cool -> 1, etc.  
For this approach, it is important that the categories can be placed in a one-dimensional order, for example:

*cold < cool < mild < warm* becomes *0 < 1 < 2 < 3*

But label encoding is only advisable in cases when the model can rightfully assume that the numbers represent an interval scale. Otherwise it will overinterpret the numbers. Then ordinal variables must like nominal variables be one-hot encoded.

#### One-Hot Encoding
In one-hot encoding each category gets its own column, filled with 1 or 0 depending on whether the category applies or not.

In [17]:
features_raw = feature_collection.copy()

# Label-Encoding für ordinale Kategorien
label_encoders = {}
for col in ["Temperatureclass", "Cloudclass", "Windclass"]:
    le = LabelEncoder()
    features_raw[col + "_enc"] = le.fit_transform(features_raw[col])
    label_encoders[col] = le

# Mapping und One-Hot-Encoding für Warengruppe
mapping = {1: 'Brot', 2: 'Broetchen', 3: 'Croissant', 4: 'Konditorei', 5: 'Kuchen', 6: 'Saisonbrot'}
features_raw['Product'] = features_raw['Warengruppe'].map(mapping)
features_encoded = pd.get_dummies(features_raw, columns=["Product"], prefix="Group")
dummy_cols = [col for col in features_encoded.columns if col.startswith("Group_")]
features_encoded[dummy_cols] = features_encoded[dummy_cols].astype(int)

# One-Hot-Encoding für Weathercategory
features_encoded = pd.get_dummies(features_encoded, columns=["Weathercategory"], prefix="Weather")
weather_columns = [col for col in features_encoded.columns if col.startswith("Weather_")]
features_encoded[weather_columns] = features_encoded[weather_columns].astype(int)

# One-Hot-Encoding für Wochentag
features_encoded = pd.get_dummies(features_encoded, columns=["Wochentag"], prefix="Weekday")
weekday_cols = [col for col in features_encoded.columns if col.startswith("Weekday_")]
features_encoded[weekday_cols] = features_encoded[weekday_cols].astype(int)

# One-Hot-Encoding für Monat
features_encoded = pd.get_dummies(features_encoded, columns=["Monat"], prefix="Month")
month_cols = [col for col in features_encoded.columns if col.startswith("Month_")]
features_encoded[month_cols] = features_encoded[month_cols].astype(int)

### 3.2 Clean Up
Drop all columns that schould not be in the trainings dataset.

In [18]:
features = features_encoded.copy()
features = features.drop(columns=['Warengruppe', 'Wettercode', 'Month', 'Quarter', 'Temperatureclass', 'Cloudclass', 'Windclass'])
pd.set_option('display.max_columns', None)
features.info()
features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11164 entries, 0 to 11163
Data columns (total 57 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   id                                 11164 non-null  int64         
 1   Datum                              11164 non-null  datetime64[ns]
 2   Umsatz                             9334 non-null   float64       
 3   Bewoelkung                         11164 non-null  float64       
 4   Temperatur                         11164 non-null  float64       
 5   Windgeschwindigkeit                11164 non-null  float64       
 6   Rain                               11164 non-null  float64       
 7   KielerWoche                        11164 non-null  int64         
 8   CPI                                11164 non-null  float64       
 9   Unemployment                       11164 non-null  float64       
 10  GDP                               

,id,Datum,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Rain,KielerWoche,CPI,Unemployment,GDP,Rain_Sum_8to18,Rain_8to18,Temp_5d_avg,Temp_above_avg,Temp_below_avg,Tmax,Tmax_gt25,Hot_HeavyRain,Wochenende,Feiertag,Schulferien,Residual,Temperatureclass_enc,Cloudclass_enc,Windclass_enc,Group_Broetchen,Group_Brot,Group_Croissant,Group_Konditorei,Group_Kuchen,Group_Saisonbrot,Weather_After_Rain,Weather_Fog,Weather_Other,Weather_Rain,Weather_Showers_and_Thunderstorms,Weather_Snow_and_Ice,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,1307011,2013-07-01,148.828353,6.0,17.8375,15.0,1.567431,0,93.5,6.8,728.23,1.442231,1,14.4575,1,0,18.199883,0,0,0,0,1,0,2,1,2,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1307021,2013-07-02,159.793757,3.0,17.3125,10.0,0.140967,0,93.5,6.8,728.23,0.140967,0,15.4100,0,0,18.319065,0,0,0,0,1,0,2,1,2,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1307031,2013-07-03,111.885594,7.0,21.0750,6.0,6.276774,0,93.5,6.8,728.23,5.542491,1,17.0150,1,0,22.473013,0,0,0,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1307041,2013-07-04,168.864941,7.0,18.8500,7.0,0.126180,0,93.5,6.8,728.23,0.126180,0,17.8175,0,0,19.594244,0,0,0,0,1,0,2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1307051,2013-07-05,171.280754,5.0,19.9750,12.0,1.255988,0,93.5,6.8,728.23,1.255988,1,19.0100,0,0,20.406262,0,0,0,0,1,0,2,1,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11159,1812226,2018-12-22,NaN,8.0,4.3000,4.0,0.392234,0,98.5,4.9,881.52,0.243416,0,4.7575,0,0,5.505522,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
11160,1812236,2018-12-23,NaN,7.0,6.4500,9.0,4.244212,0,98.5,4.9,881.52,1.981295,1,4.9850,0,0,6.818994,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
11161,1812246,2018-12-24,NaN,7.0,2.5000,10.0,0.000000,0,98.5,4.9,881.52,0.000000,0,4.8500,0,0,4.503302,0,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11162,1812276,2018-12-27,NaN,7.0,7.1250,12.0,0.000000,0,98.5,4.9,881.52,0.000000,0,6.0700,0,0,7.492173,0,0,0,0,1,0,1,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [19]:
# Save the result to a CSV file
features.to_csv("features.csv", index=False)